In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import pandas_profiling

Start from the last cleaned file

In [3]:
data = pd.read_csv('data/Cleaned_StateData_1.csv')
data = data.fillna(0)
len(data), len(data.keys())

(2040, 583)

In [4]:
fields_separation = pd.read_csv('data/field_separation.csv')
fields_separation = fields_separation[['Attribute', 'Category', 'Category Code', 'Property']]
# fields_separation = fields_separation.dropna()
fields_separation = fields_separation.set_index('Attribute')
print(len(fields_separation), len(fields_separation.keys()))
fields_separation.head()

594 3


,Category,Category Code,Property
Attribute,,,
SurveyYr,NaN,NaN,NaN
Year4,NaN,NaN,NaN
YearofData,NaN,NaN,NaN
ID,NaN,NaN,NaN
IDChanged,NaN,NaN,NaN


In [5]:
# Columns with 100% missing values
reject = data.sum()[data.astype(bool).sum()==0].index.tolist()
print(len(reject))
reject

62


['State_IGR_Tax_Relief',
 'State_IGR_Oth_Gen_Sup',
 'State_IGR_Gen_Sup',
 'State_IGR_Health___Hos',
 'State_IGR_Highways',
 'State_IGR_Transit_Sub',
 'State_IGR_Hous_Com_Dev',
 'State_IGR_Public_Welf',
 'State_IGR_Sewerage',
 'State_IGR_Other',
 'Local_IGR_InterSchool_Aid',
 'Emp_Ret_Other_Earnings',
 'Air_Trans_IG_To_State',
 'Elem_Educ_IG_To_State',
 'Higher_Ed_IG_To_St',
 'Educ_NEC_IG_To_State',
 'Fin_Admin_IG_To_State',
 'Fire_Prot_IG_To_State',
 'Fire_Prot_IG_Local_Govts',
 'Judicial_IG_To_State',
 'Cen_Staff_IG_To_State',
 'Health_IG_To_State',
 'Total_Hospital_IG_To_State',
 'Hosp_Other_IG_To_State',
 'Regular_Hwy_IG_To_Sta',
 'Transit_Sub_IG_To_Sta',
 'Hous___Com_IG_To_State',
 'Libraries_IG_To_State',
 'Natural_Res_IG_To_Sta',
 'Parking_IG_To_State',
 'Parks___Rec_IG_To_Sta',
 'Police_Prot_IG_To_Sta',
 'Prot_Insp_IG_To_State',
 'Welf_Categ_IG_To_State',
 'Welf_NEC_IG_To_State',
 'Sewerage_IG_To_State',
 'SW_Mgmt_IG_To_State',
 'Water_Trans_IG_To_Sta',
 'General_NEC_IG_To_St',


In [6]:
print(len(data.keys()))
data = data.drop(columns=reject)
print(len(data.keys()))

583
521


In [7]:
print('Percentage of data in each column')
counts = (data.astype(bool).sum().sort_values()/len(data))*100
counts

Percentage of data in each column


State_IGR_Education          0.049020
LTD_Ret_FFC_Elec_Util        0.049020
LTD_Out_FFC_Elec_Util        0.049020
Total_State_IG_Revenue       0.049020
LTD_Iss_FFC_Elec_Util        0.049020
Prop_Sale_Hous_Com_Dev       0.049020
Correct_IG_To_St             0.098039
LTD_Iss_Util_Gas_Supply      0.098039
IG_Exp_To_State_Govt         0.098039
LTD_Iss_NG_Gas_Util          0.098039
Beg_LTD_Out_Trans_Util       0.147059
Beg_LTD_Out_Gas_Util         0.147059
Parking_IG_Local_Govts       0.686275
Beg_LTD_Out_Elec_Util        0.735294
LTD_Iss_FFC_Trans_Util       0.735294
LTD_Ret_NG_Gas_Util          0.784314
LTD_Ret_Util_Gas_Supply      0.784314
LTD_Out_Util_Gas_Supply      0.833333
LTD_Out_NG_Gas_Util          0.833333
Gas_Util_Cap_Outlay          0.833333
Gas_Util_Construct           0.833333
Beg_LTD_Out_Water_Util       1.078431
Parking_Construction         1.078431
LTD_Iss_NG_Trans_Util        1.078431
LTD_Iss_Util_Transit         1.078431
Gas_Util_Inter_Exp           1.127451
LTD_Out_FFC_

In [8]:
bin_size = 5
for i in np.arange(0, 100, bin_size):
    print(len(counts[(counts>i) & (counts<=i+bin_size)].index), 'columns have (' + str(i) + ',' + str(i+bin_size) + '] % data')

65 columns have (0,5] % data
27 columns have (5,10] % data
14 columns have (10,15] % data
15 columns have (15,20] % data
15 columns have (20,25] % data
3 columns have (25,30] % data
18 columns have (30,35] % data
10 columns have (35,40] % data
5 columns have (40,45] % data
15 columns have (45,50] % data
16 columns have (50,55] % data
16 columns have (55,60] % data
8 columns have (60,65] % data
31 columns have (65,70] % data
10 columns have (70,75] % data
7 columns have (75,80] % data
12 columns have (80,85] % data
9 columns have (85,90] % data
25 columns have (90,95] % data
200 columns have (95,100] % data


Lets check the 9 columns with 85-90% data

In [9]:
keys = counts[(counts>85) & (counts<=90)].index
keys

Index(['LTD_Out_Private_Purp', 'Public_Utility_Tax',
       'Unemp_Comp_Other_Balance', 'Air_Trans_Total_Expend',
       'Individual_Income_Tax', 'Unemp_Ext___Spec_Pmts',
       'Libraries_Cap_Outlay', 'Total_Hospital_Construct',
       'Unemp_Int_Revenue'],
      dtype='object')

In [26]:
for key in keys:
    df = pd.pivot_table(data, index='Year', columns='State', values=key).astype(bool).sum(axis=1)
    l = df[df==0].keys().tolist()
    if len(l) > 0:
        print('Column:', key.upper())
        print('Years:', l)

Column: LTD_OUT_PRIVATE_PURP
Years: [1977, 1978, 1979, 1980, 1981]


In [81]:
# For following columns there is no data for the following years:
d = {}
for key in set(data.keys())-set(['State', 'Year']):
    df = pd.pivot_table(data, index='Year', columns='State', values=key).astype(bool).sum(axis=1)
    l = df[df==0].keys().tolist()
    if len(l) > 0:
        print('Column:', key.upper(), '(', len(l), 'years )')
        print('Years:', l)
        d[key] = len(l)

Column: LTD_RET_NG_UTILITY ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_RET_NG_TRANS_UTIL ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_OUT_NG_ELEC_UTIL ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_RET_NG_ELEM_EDUC ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_OUT_NG_GENERAL ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_OUT_FFC_GENERAL ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: ELEM_EDUC_IG_SCH_TO_SCH ( 30 years )
Years: [1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006]
Column: LTD_ISS_NG_WATER_UTIL ( 16 year

Column: LTD_OUT_NG_OTHER_NEC ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: WELF_STATE_SHARE_PART_D ( 39 years )
Years: [1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_ISS_GENERAL ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_RET_UTIL_TRANSIT ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_OUT_FFC_ELEM_EDUC ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_OUT_UTIL_GAS_SUPPLY ( 23 years )
Years: [1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1989, 1990, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: BEG_LTD_OUT_GENERAL ( 18 years

Column: LTD_RET_FFC_TRANS_UTIL ( 14 years )
Years: [1982, 1983, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LOCAL_IGR_HOUS_COM_DEV ( 29 years )
Years: [1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005]
Column: GAS_UTIL_INTER_EXP ( 17 years )
Years: [1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1987, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_OUT_NG_TRANS_UTIL ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_RET_UTIL_GAS_SUPPLY ( 24 years )
Years: [1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1989, 1990, 2000, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_RET_FFC_WATER_UTIL ( 12 years )
Years: [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Column: LTD_OUT_FFC_WATER_UTIL ( 12 years )


In [82]:
# There are n states who no state provided any data 
missing_num_years = pd.DataFrame.from_dict(data=d, orient='index', columns=['num']).sort_values(by='num')
missing_num_years#.loc[[index for index in missing_num_years.index if 'IGR' in index]]

,num
Emp_Retire_Sec_Corp_Stk,1
Emp_Retire_Sec_Corp_Bds,1
Emp_Retire_Sec_Mortgages,1
Local_IGR_Oth_Gen_Sup,1
Chg_Elem_Ed_Tuition,2
Parking_Total_Expend,2
Liquor_Stores_Constr,2
Parking_Direct_Expend,2
Parking_Current_Exp,2
Fire_Prot_Construction,3


In [72]:
pd.pivot_table(data, index='State', columns='Year', values='Local_IGR_Highways').astype(bool).sum(axis=1)


State
ALABAMA               40
ALASKA                 4
ARIZONA               39
ARKANSAS              26
CALIFORNIA            39
COLORADO              40
CONNECTICUT            9
DELAWARE               1
FLORIDA               17
GEORGIA               40
HAWAII                 3
IDAHO                 40
ILLINOIS              40
INDIANA               40
IOWA                  40
KANSAS                40
KENTUCKY              21
LOUISIANA             16
MAINE                 40
MARYLAND              39
MASSACHUSETTS         36
MICHIGAN              40
MINNESOTA             40
MISSISSIPPI           29
MISSOURI              37
MONTANA               33
NEBRASKA              40
NEVADA                40
NEW HAMPSHIRE         40
NEW JERSEY            40
NEW MEXICO            15
NEW YORK              12
NORTH CAROLINA        40
NORTH DAKOTA          40
OHIO                  40
OKLAHOMA              40
OREGON                29
PENNSYLVANIA          40
RHODE ISLAND          13
SOUTH CAROLINA     

In [79]:
len(fields_separation['Property'].drop_duplicates())

186

In [66]:
data.keys().tolist()

['Year',
 'ID',
 'State_Code',
 'State',
 'FIPS_Code_State',
 'FYEndDate',
 'Population',
 'Total_Revenue',
 'Total_Rev_Own_Sources',
 'General_Revenue',
 'Gen_Rev_Own_Sources',
 'Total_Taxes',
 'Property_Tax',
 'Tot_Sales___Gr_Rec_Tax',
 'Total_Gen_Sales_Tax',
 'Total_Select_Sales_Tax',
 'Alcoholic_Beverage_Tax',
 'Amusement_Tax',
 'Insurance_Premium_Tax',
 'Motor_Fuels_Tax',
 'Pari_mutuels_Tax',
 'Public_Utility_Tax',
 'Tobacco_Tax',
 'Other_Select_Sales_Tax',
 'Total_License_Taxes',
 'Alcoholic_Beverage_Lic',
 'Amusement_License',
 'Corporation_License',
 'Hunting___Fishing_License',
 'Motor_Vehicle_License',
 'Motor_Veh_Oper_License',
 'Motor_Vehicle_License_Total',
 'Public_Utility_License',
 'Occup_and_Bus_Lic_NEC',
 'Other_License_Taxes',
 'Total_Income_Taxes',
 'Individual_Income_Tax',
 'Corp_Net_Income_Tax',
 'Death_and_Gift_Tax',
 'Docum_and_Stock_Tr_Tax',
 'Severance_Tax',
 'Taxes_NEC',
 'Total_IG_Revenue',
 'Total_Fed_IG_Revenue',
 'Fed_IGR_Air_Transport',
 'Fed_IGR_Educati

In [64]:
# For following columns there is no data from the following states:
d = {}
for key in set(data.keys())-set(['State', 'Year']):
    df = pd.pivot_table(data, index='State', columns='Year', values=key).astype(bool).sum(axis=1)
    l = df[df==0].keys().tolist()
    if len(l) > 0:
        print('Column:', key.upper(), '(', len(l), 'states )')
        print('States:', l)
        d[key] = len(l)

Column: LTD_RET_NG_UTILITY ( 40 states )
States: ['ALABAMA', 'ARKANSAS', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEW HAMPSHIRE', 'NEW MEXICO', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OREGON', 'PENNSYLVANIA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']
Column: WELF_CASH_CASH_ASSIST ( 2 states )
States: ['COLORADO', 'NEW YORK']
Column: LTD_RET_NG_TRANS_UTIL ( 49 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', '

Column: PARKING_CAPITAL_OUTLAY ( 44 states )
States: ['ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'IDAHO', 'ILLINOIS', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']
Column: LTD_RET_FFC_UTILITY ( 46 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEW HAMPSHIRE', 'NEW MEXICO', 'NORTH CAROLINA', '

Column: CHG_SEWERAGE ( 26 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'KANSAS', 'LOUISIANA', 'MAINE', 'MICHIGAN', 'MISSISSIPPI', 'MISSOURI', 'NEBRASKA', 'NEVADA', 'NEW MEXICO', 'NORTH DAKOTA', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'VIRGINIA', 'WYOMING']
Column: DOCUM_AND_STOCK_TR_TAX ( 15 states )
States: ['ALASKA', 'CALIFORNIA', 'COLORADO', 'IDAHO', 'INDIANA', 'KANSAS', 'LOUISIANA', 'MISSISSIPPI', 'MONTANA', 'NEW MEXICO', 'NORTH DAKOTA', 'OHIO', 'TEXAS', 'UTAH', 'WYOMING']
Column: LTD_ISS_FFC_WATER_UTIL ( 48 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEW HAMPSHIRE', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 

Column: LTD_ISS_NG_GAS_UTIL ( 50 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WASHINGTON DC CITY', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']
Column: GAS_UTIL_CONSTRUCT ( 50 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 

Column: EDUC_NEC_IG_LOCAL_GOVTS ( 1 states )
States: ['WASHINGTON DC CITY']
Column: LOCAL_IGR_SEWERAGE ( 37 states )
States: ['ALABAMA', 'ARKANSAS', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OREGON', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TEXAS', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']
Column: FED_IGR_TRANSIT_SUB ( 18 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'IDAHO', 'KANSAS', 'LOUISIANA', 'MISSISSIPPI', 'NEBRASKA', 'NEVADA', 'NORTH DAKOTA', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'UTAH', 'WASHINGTON', 'WYOMING']
Column: TRANS_UTIL_TOTAL_EXP ( 9 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'COLORADO', 'IDAHO', 'KANSAS', 'NEVADA', 'SOUTH CAROLI

Column: LTD_OUT_UTIL_ELECTRIC ( 44 states )
States: ['ALABAMA', 'ARKANSAS', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WASHINGTON DC CITY', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']
Column: HOSP_OTHER_CONSTRUCT ( 36 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'CALIFORNIA', 'COLORADO', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'MICHIGAN', 'MINNESOTA', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSE

Column: TRANS_UTIL_CURRENT_EXP ( 9 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'COLORADO', 'IDAHO', 'KANSAS', 'NEVADA', 'SOUTH CAROLINA', 'SOUTH DAKOTA']
Column: GAS_UTIL_TOTAL_EXP ( 48 states )
States: ['ALABAMA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH DAKOTA', 'OHIO', 'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WASHINGTON DC CITY', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']
Column: LTD_RET_GEN_ELEM_EDUC ( 28 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'COLORADO', 'FLORIDA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'MASSA

Column: LTD_RET_FFC_TRANS_UTIL ( 49 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WASHINGTON DC CITY', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']
Column: LOCAL_IGR_HOUS_COM_DEV ( 42 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MISSISSIPPI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW 

Column: EMP_RET_OTHER_PAYMTS ( 1 states )
States: ['WASHINGTON DC CITY']
Column: LTD_RET_FFC_ELEM_EDUC ( 37 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'COLORADO', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MASSACHUSETTS', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VIRGINIA', 'WASHINGTON DC CITY', 'WISCONSIN', 'WYOMING']
Column: ST_DEBT_END_OF_YEAR ( 1 states )
States: ['WYOMING']
Column: LOCAL_IGR_TRANSIT_SUB ( 30 states )
States: ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'HAWAII', 'IDAHO', 'ILLINOIS', 'IOWA', 'KANSAS', 'MAINE', 'MISSISSIPPI', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW MEXICO', 'NORTH DAKOTA', 'OHIO', 'OREGON', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TEXAS', 'UTAH', 'VERMONT', 'VIRG

Column: LIQUOR_STORES_CONSTR ( 39 states )
States: ['ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MINNESOTA', 'MISSOURI', 'NEBRASKA', 'NEVADA', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'WASHINGTON DC CITY', 'WEST VIRGINIA', 'WISCONSIN']
Column: SW_MGMT_DIRECT_EXPEND ( 2 states )
States: ['ALASKA', 'SOUTH DAKOTA']
Column: TOTAL_UTIL_CONSTRUCT ( 14 states )
States: ['ALABAMA', 'ARKANSAS', 'IDAHO', 'IOWA', 'KANSAS', 'MAINE', 'NEBRASKA', 'NEW MEXICO', 'OHIO', 'PENNSYLVANIA', 'SOUTH DAKOTA', 'TENNESSEE', 'UTAH', 'VIRGINIA']
Column: PARKING_CONSTRUCTION ( 46 states )
States: ['ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 'COLORADO', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'IDAHO', 'I

In [65]:
missing_num_states = pd.DataFrame.from_dict(data=d, orient='index', columns=['num']).sort_values(by='num')
missing_num_states

,num
Welf_Cash_Total_Exp,1
Total_Educ_Assist___Sub,1
Elem_Educ_IG_Local_Govts,1
Beg_LTD_Out_Priv_Purp,1
Educ_NEC_Direct_Expend,1
Alcoholic_Beverage_Lic,1
Regular_Hwy_IG_Loc_Govts,1
Welf_NEC_IG_Local_Govts,1
LTD_Out_Gen_Other_Educ,1
Educ_NEC_Construction,1


In [45]:
'State_IGR_Education', 'Prop_Sale_Hous_Com_Dev', 'Total_State_IG_Revenue', 'LTD_Out_FFC_Elec_Util', 'LTD_Ret_FFC_Elec_Util', 'LTD_Iss_FFC_Elec_Util', 'Welf_State_Share_Part_D'

('State_IGR_Education',
 'Prop_Sale_Hous_Com_Dev',
 'Total_State_IG_Revenue',
 'LTD_Out_FFC_Elec_Util',
 'LTD_Ret_FFC_Elec_Util',
 'LTD_Iss_FFC_Elec_Util',
 'Welf_State_Share_Part_D')

In [ ]:
props = fields_separation['Property'].dropna().drop_duplicates().sort_values().values.tolist()
props

In [ ]:
fields_separation.keys()

In [ ]:
prop = props[0]
prop

In [ ]:
fields_separation[fields_separation['Property']==prop]['Attribute'].values.tolist()

In [ ]:
dict_props = {}
for prop in props:
    dict_props[prop] = [fields_separation[fields_separation['Property']==prop]['Attribute'].values.tolist()]
dict_props

In [ ]:
df = pd.DataFrame(data=dict_props).T
df

In [ ]:
df['len'] = df[0].str.len()
df = df.rename(columns={0:'list'})
df = df.sort_values(by='len', ascending=False)
df

In [ ]:
props = df.index.tolist()
props

In [ ]:
[key for key in fields_separation['Attribute'] if key not in data.keys()]

In [ ]:
for prop in props:
    print(prop)
    data[df.loc[prop]['list']]

In [ ]:
prop = props[0]
pandas_profiling.ProfileReport(data[df.loc[prop]['list']])

In [ ]:
key = 'Fed_IGR_Education'

In [ ]:
plt.scatter(x=data['State'], y=data['Year'], s=2*(data[key]-data[key].mean())/data[key].std())
plt.xticks(rotation=90)
plt.show()

In [ ]:
key = 'Fed_IGR_Education'
plt.title(key)
for state in data['State'].drop_duplicates():
    x = data[data['State']==state]
    plt.plot(x['Year'], x[key])
plt.show()

In [ ]:
keys = dict_props['Education'][0]
for key in keys:
    plt.title(key)
    for state in data['State'].drop_duplicates():
        x = data[data['State']==state]
        plt.plot(x['Year'], x[key])
    plt.show()

In [ ]:
count = 0
for key in set(data.keys()) - set(['State', 'Year']):
    df = pd.pivot_table(data, index='State', columns='Year', values=key).astype(bool).sum(axis=1)
    df = df[df==0].sort_values()
    if len(df) >= 50:
        data.drop(columns=key)
        continue
    if len(df) > 0:
        count += 1
        print(key, len(df))
        print(df)
        

In [ ]:
count

In [ ]:
len(data.keys())

In [ ]:
for key in data.keys():
    df = pd.pivot_table(data, index='Year', columns='State', values='Fed_IGR_Education').astype(bool).sum(axis=1)
    df = df[df<40]
    if len(df) > 0:
        print(key)
        print(df)

In [ ]:
plt.matshow(corr.abs()>0.95)

In [ ]:
plt.isinteractive()

In [ ]:
plt.imshow(np.asarray(corr))

In [ ]:
prop = props[0]
prop

In [ ]:
data[dict_props[prop]].astype(bool).sum()/len(data)

In [ ]:

data[dict_props[props[3]]].corr().iloc[0]